In [8]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from numpy.core.umath_tests import inner1d
from sklearn.externals import joblib

C:\Users\Aniket\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  


In [4]:
import joblib

In [10]:
cleanedDF = pd.read_csv('../data/processed/match_cleaned.csv')
playerDF = pd.read_csv('../data/processed/player.csv')
bowlerwiseDF = pd.read_csv('../data/processed/bowlerwise.csv')
teamsDF = pd.read_csv('../data/Team.csv')
venueDF = pd.read_csv('../data/venue.csv')

    
def Sort_Tuple(tup):  
  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    return(sorted(tup, key = lambda x: x[1],reverse=1))   
  

In [11]:
def feature_fetch(season,team1, team2):
   # print(playerDF['is_bowler'])
    t1df=playerDF[ playerDF['Team_'+str(int(season))]==int(team1)]
    t2df=playerDF[ playerDF['Team_'+str(int(season))]==int(team2)]
    t1playersid=list(t1df.loc[:,'Player_Id'])
    t1playersm=list(t1df.loc[:,"MPlayed_"+str(int(season-1))])
    t2playersid=list(t2df.loc[:,'Player_Id'])
    t2playersm=list(t2df.loc[:,"MPlayed_"+str(int(season-1))])
    t1bowlerid=list(t1df[t1df['is_bowler']==1].loc[:,'Player_Id'])
    t1bowlerm=list(t1df[t1df['is_bowler']==1].loc[:,"MPlayed_"+str(int(season-1))])
    t2bowlerid=list(t2df[t2df['is_bowler']==1].loc[:,'Player_Id'])
    t2bowlerm=list(t2df[t2df['is_bowler']==1].loc[:,"MPlayed_"+str(int(season-1))])
    #print(t2playersid,t2playersm)
    t1 = list(zip(([int(x) for x in t1playersid]),([int(x) for x in t1playersm])))
    t2 = list(zip(([int(x) for x in t2playersid]),([int(x) for x in t2playersm])))
    t1b = list(zip(([int(x) for x in t1bowlerid]),([int(x) for x in t1bowlerm])))
    t2b = list(zip(([int(x) for x in t2bowlerid]),([int(x) for x in t2bowlerm])))
    t1=Sort_Tuple(t1)
    t2=Sort_Tuple(t2)
    t1b=Sort_Tuple(t1b)
    t2b=Sort_Tuple(t2b)
        
    t1pxi=t1b[0:5]
    [t1.remove(x) for x in t1pxi]
    t1pxi+=t1[0:6]
    t2pxi=t2b[0:5]
    [t2.remove(x) for x in t2pxi]
    t2pxi+=t2[0:6]
    t1pxi=[x[0] for x in t1pxi]
    t2pxi=[x[0] for x in t2pxi]
    #print(t1pxi,t2pxi)
    runst1=[]
    runst2=[]
    for i in range(11):
        pDF=bowlerwiseDF[bowlerwiseDF['Player_Id']==t1pxi[i]]
        runs1=[]
        for j in range(11):
            runs1+=list(zip(list(pDF.loc[:,"B"+str(t2pxi[j])+"runs"]),list(list(pDF.loc[:,"B"+str(t2pxi[j])+"balls"]))))
        runst1+=(runs1)
    for i in range(11):
        pDF=bowlerwiseDF[bowlerwiseDF['Player_Id']==t2pxi[i]]
        runs2=[]
        for j in range(11):
            runs2+=list(zip(list(pDF.loc[:,"B"+str(t1pxi[j])+"runs"]),list(list(pDF.loc[:,"B"+str(t1pxi[j])+"balls"]))))
        runst2+=(runs2)
    return [runst1,runst2]

In [12]:
#feature_fetch(2017,1,2)

In [13]:
def features():
    print("Generating Features")
    data = pd.DataFrame()
    pos = 0
    totalSize=len(cleanedDF)
    for index,row in cleanedDF.iterrows():
        print("Processing data...( %d %% done)" % int(pos*100/totalSize) )
        matchId =int( row['match_id'])
        data.loc[pos,"matchId"] = matchId
        Season = int(row['Season_Year'])
        data.loc[pos,"Season_Year"] = Season
        team1 = int(row['Team1'])
        data.loc[pos,"team1"] = team1
        team2 = int(row['Team2'])
        data.loc[pos,"team2"] = team2
        City_Name = int(row['City_Name'])
        data.loc[pos,"City_Name"] = City_Name
        Toss_Winner = int(row['Toss_Winner'])
        data.loc[pos,"Toss_Winner"] = 0 if Toss_Winner==team1 else 1
        Toss_Name = int(row['Toss_Name'])
        data.loc[pos,"Toss_Name"] = Toss_Name

        
        a=0
        for i in feature_fetch(Season,team1,team2):
            a+=1
            b=0
            c=[]
            for j in i:
                b+=1
                c.append(j[0]/j[1] if j[1] else 0)
            data.loc[pos,"feature"+str(b)+"runs"+str(a)] = sum(c)/len(c)
        match_winner = int(row['match_winner'])
        data.loc[pos,"match_winner"] = 0 if match_winner==team1 else 1
        pos=pos+1
    #data.to_csv("../data/matchbowlerwise.csv",sep=',')
    print("Done")

In [14]:
#features()

In [15]:
matchbowlerwiseDF = pd.read_csv('../data/matchbowlerwise.csv')

In [16]:
def predict(season,team1,team2,city,tw,tn,rf):
    
    data=[tw,tn,city]
    a=0
    for i in feature_fetch(season,team1,team2):
        a+=1
        b=0
        c=[]
        for j in i:
            b+=1
            c.append(j[0]/j[1] if j[1] else 0)
        #print(sum(c)/len(c))
        data.append(sum(c)/len(c))
    #print(data)
    predictions = rf.predict([data])
    return team1 if predictions[0]==0 else team2

In [17]:
def return_entry():
    try:
        season = E1.get()
        team1=getTeamId(E2.get())
        team2=getTeamId(E3.get())
        toss_winner=getTeamId(E4.get())
        toss_decision=getTossDecision(E5.get())
        city=getVenueId(E6.get())
        arr=[season,team1,team2,city,toss_winner,toss_decision]
        print(arr)
        L0.configure(text = "Winner: " +str(getTeamName(predict(int(season),int(team1),int(team2),int(city),int(toss_winner),int(toss_decision),loaded_model))))
    except:
        L0.configure(text = "Wrong Details ")
    
def getTeamId(name):
    return (list((teamsDF[(teamsDF["Short_Form"] == name)]).loc[:,'Team_Id'])[0]) if len(list((teamsDF[(teamsDF["Short_Form"] == name)]).loc[:,'Team_Id'])) else 0


def getVenueId(name):
    return (list((venueDF[(venueDF["City_Name"] == name)]).loc[:,'vid'])[0]) if len(list((venueDF[(venueDF["City_Name"] == name)]).loc[:,'vid'])) else 0


def getTossDecision(name):
    if ( name=="BAT"):
        return 0
    else:
        return 1
    
    
def getTeamName(name):
    return (list((teamsDF[(teamsDF["Team_Id"] == name)]).loc[:,'Short_Form'])[0]) if len(list((teamsDF[(teamsDF["Team_Id"] == name)]).loc[:,'Short_Form'])) else 0

In [18]:
def training4():
    array=matchbowlerwiseDF.values
    x=array[:,5:10]
    y=array[:,10]
    #print(x)
    #print(y)
    validation_size = 0.75
    seed = 7
    x_train, x_validation, y_train, y_validation = model_selection.train_test_split(x, y, test_size=validation_size,
                                                                                    random_state=seed)
    
    print("Random Forest")
    rf = RandomForestClassifier()
    rf.fit(x_train, y_train)
    accu = rf.score(x_validation,y_validation)
    print(accu)
    #predictions = rf.predict([results])
    #print("model->" + convert_to_team(predictions))
    return rf;

In [23]:
rf4=training4()

Random Forest
0.5355648535564853


C:\Users\Aniket\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [25]:
filename = 'finalized_model.sav'
filename1 = 'finalized_model1.sav'
filename2 = 'finalized_model2.sav'
#joblib.dump(rf4, filename)

In [26]:
loaded_model = joblib.load(filename)
loaded_model1 = joblib.load(filename1)
loaded_model2 = joblib.load(filename2)

C:\Users\Aniket\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.2 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Aniket\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.2 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [27]:
#print(rf4)

In [28]:
rf4=training4()

Random Forest
0.5209205020920502


C:\Users\Aniket\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
matches_cleaned_data = pd.read_csv('../data/processed/test_2019_cleaned.csv')
matches_df = matches_cleaned_data[['Season_Year','Team1', 'Team2', 'City_Name', 'Toss_Winner', 'Toss_Name', 'match_winner']]
array = matches_df.values
x = array[:, 0:6]
x=list(x)
for i in range(len(x)):
    x[i]=list(x[i])

y = array[:, 6]
data = pd.DataFrame()
pos=0
total=0
correct=0
for t in range(0,len(x)):
    #print (t);
    data.loc[pos,"Team1"] = getTeamName(x[t][1])
    data.loc[pos,"Team2"] = getTeamName(x[t][2])
    data.loc[pos,"Winner"] = getTeamName(y[t])
    temp1=predict((x[t][0]),(x[t][1]),(x[t][2]),(x[t][3]),(x[t][4]),(x[t][5]),loaded_model2)
    #temp1=predict((x[t][0]),(x[t][1]),(x[t][2]),(x[t][3]),(x[t][4]),(x[t][5]),rf4)
    #temp2=predict((x[t][0]),(x[t][1]),(x[t][2]),(x[t][3]),(x[t][4]),(x[t][5]),loaded_model2)
    #temp3=predict((x[t][0]),(x[t][1]),(x[t][2]),(x[t][3]),(x[t][4]),(x[t][5]),loaded_model2)
    #if(temp1==temp2):
    #    temp=temp2
    #else:
    #    temp=predict((x[t][0]),(x[t][1]),(x[t][2]),(x[t][3]),(x[t][4]),(x[t][5]),loaded_model)
    #print(int(temp),int(y[t]))    
    temp=temp1
    data.loc[pos,"Prediction"] = getTeamName(temp)    
    total=total+1
    pos=pos+1
    if(y[t]==temp):
        correct=correct+1
        
data.to_csv("../data/processed/report_final_2019_temp.csv",sep=',')
print("correct=",correct)
print("total=",total)

correct= 42
total= 59


In [30]:
loaded_model2

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [31]:
ft_weights = pd.DataFrame(loaded_model2.feature_importances_)

In [32]:
ft_weights

,0
0,0.202230
1,0.055788
2,0.059012
3,0.337895
4,0.345076


In [33]:
from tkinter import *
top=Tk()
O1=["CSK","RCB","KKR"]

L0=Label(top)
L0.grid(row=0,column=1)
L1 = Label(top, text="Season").grid(row=1,column=0)
E1 = Entry(top,bd=5)
E1.grid(row=1,column=1)

L2 = Label(top, text="Team 1").grid(row=2,column=0)
E2 = Entry(top, bd =5 )
E2.grid(row=2,column=1)

L3 = Label(top, text="Team 2").grid(row=3,column=0)
E3 = Entry(top, bd =5)
E3.grid(row=3,column=1)

L4 = Label(top, text="Toss Winner").grid(row=4,column=0)
E4 = Entry(top, bd =5)
E4.grid(row=4,column=1)

L5 = Label(top, text="Toss Decision").grid(row=5,column=0)
E5 = Entry(top, bd =5)
E5.grid(row=5,column=1)

L6 = Label(top, text="City").grid(row=6,column=0)
E6 = Entry(top, bd =5)
E6.grid(row=6,column=1)

B1=Button(top,text="Submit",command = return_entry)
B1.grid(row=7,column=1)

#B2=Button(top,text="Quit",command = top.quit)
#B2.grid(row=8,column=1)

top.mainloop()


['2018', 7, 3, 4, 3, 0]
['2018', 7, 3, 4, 3, 1]
